In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
import torch
import torchvision
import torch.nn

device = 'cuda' if torch.cuda.is_available() else 'cpu'
# 랜덤 시드 고정
torch.manual_seed(777)
# GPU 사용 가능일 경우 랜덤 시드 고정
if device == 'cuda':
    torch.cuda.manual_seed_all(777)

#CSV 파일 불러오기

In [ ]:
import pandas as pd

file_path1 = '/content/gdrive/MyDrive/MAIC2023_공모전/ECG_child_age_train.csv'
file_path2 = '/content/gdrive/MyDrive/MAIC2023_공모전/ECG_adult_age_train.csv'
file_path3 = '/content/gdrive/MyDrive/MAIC2023_공모전/submission.csv'

# 파일 불러오기
ECG_child_age_train = pd.read_csv(file_path1)
ECG_adult_age_train = pd.read_csv(file_path2)
submission = pd.read_csv(file_path3)

# 데이터 확인
print(ECG_child_age_train.head())
print(ECG_adult_age_train.head())
print(submission.head())

      FILENAME  GENDER       AGE
0  ecg_child_0    MALE  1.500000
1  ecg_child_1    MALE  4.833333
2  ecg_child_2  FEMALE  5.583333
3  ecg_child_3  FEMALE  5.083333
4  ecg_child_4    MALE  0.166667
      FILENAME GENDER  AGE
0  ecg_adult_0   MALE   53
1  ecg_adult_1   MALE   68
2  ecg_adult_2   MALE   68
3  ecg_adult_3   MALE   89
4  ecg_adult_4   MALE   79
         FILENAME  GENDER  AGE
0  ecg_child_8781  FEMALE  NaN
1  ecg_child_8782    MALE  NaN
2  ecg_child_8783    MALE  NaN
3  ecg_child_8784  FEMALE  NaN
4  ecg_child_8785  FEMALE  NaN


In [ ]:
ECG_child_age_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8781 entries, 0 to 8780
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   FILENAME  8781 non-null   object 
 1   GENDER    8781 non-null   object 
 2   AGE       8781 non-null   float64
dtypes: float64(1), object(2)
memory usage: 205.9+ KB


In [ ]:
#ECG_adult_age_train[ECG_adult_age_train['GENDER']=='unknown']

#ECG 데이터 zip파일 압축 해제 후 디렉토리에 저장

In [ ]:
'''
import zipfile

# 압축 파일 경로들 지정
zip_files = ['/content/gdrive/MyDrive/MAIC2023_공모전/ECG_child_numpy_valid.zip',
             '/content/gdrive/MyDrive/MAIC2023_공모전/ECG_child_numpy_train.zip',
             '/content/gdrive/MyDrive/MAIC2023_공모전/ECG_adult_numpy_valid.zip',
             '/content/gdrive/MyDrive/MAIC2023_공모전/ECG_adult_numpy_train.zip']

# 원래 디렉토리 경로 지정
destination_dir = '/content/gdrive/MyDrive/MAIC2023_공모전/'

# 모든 압축 파일들을 원래 디렉토리에 압축 해제
for zip_file in zip_files:
    with zipfile.ZipFile(zip_file, 'r') as zip_ref:
        zip_ref.extractall(destination_dir)

    # 압축 파일 삭제 (필요한 경우)
    import os
    os.remove(zip_file)

# 압축 해제 완료 메시지 출력
print("모든 압축 파일을 원래 디렉토리에 압축 해제하였습니다.")
'''

'\nimport zipfile\n\n# 압축 파일 경로들 지정\nzip_files = [\'/content/gdrive/MyDrive/MAIC2023_공모전/ECG_child_numpy_valid.zip\',\n             \'/content/gdrive/MyDrive/MAIC2023_공모전/ECG_child_numpy_train.zip\',\n             \'/content/gdrive/MyDrive/MAIC2023_공모전/ECG_adult_numpy_valid.zip\',\n             \'/content/gdrive/MyDrive/MAIC2023_공모전/ECG_adult_numpy_train.zip\']\n\n# 원래 디렉토리 경로 지정\ndestination_dir = \'/content/gdrive/MyDrive/MAIC2023_공모전/\'\n\n# 모든 압축 파일들을 원래 디렉토리에 압축 해제\nfor zip_file in zip_files:\n    with zipfile.ZipFile(zip_file, \'r\') as zip_ref:\n        zip_ref.extractall(destination_dir)\n\n    # 압축 파일 삭제 (필요한 경우)\n    import os\n    os.remove(zip_file)\n\n# 압축 해제 완료 메시지 출력\nprint("모든 압축 파일을 원래 디렉토리에 압축 해제하였습니다.")\n'

In [ ]:
import os
import io
import zipfile
from zipfile import ZipFile
from io import BytesIO
from numpy.lib.type_check import common_type
import numpy as np

In [ ]:
ct_filename = []  ##파일이름 리스트
ct_data=[]        ##데이터 리스트
with ZipFile('/content/gdrive/MyDrive/MAIC2023_공모전/ECG_child_numpy_train.zip', 'r') as zipObj:
    listOfFileNames = zipObj.namelist()
    for fileName in listOfFileNames:
        if fileName.endswith('npy'):
            zipRead = zipObj.read(fileName)
            curr_df = np.load(BytesIO(zipRead))
            ct_data.append(curr_df)
            ct_filename.append(fileName)

In [ ]:
cv_filename = []  ##파일이름 리스트
cv_data=[]        ##데이터 리스트
with ZipFile('/content/gdrive/MyDrive/MAIC2023_공모전/ECG_child_numpy_valid.zip', 'r') as zipObj:
    listOfFileNames = zipObj.namelist()
    for fileName in listOfFileNames:
        if fileName.endswith('npy'):
            zipRead = zipObj.read(fileName)
            curr_df = np.load(BytesIO(zipRead))
            cv_data.append(curr_df)
            cv_filename.append(fileName)

In [ ]:
av_filename = []  ##파일이름 리스트
av_data=[]        ##데이터 리스트
with ZipFile('/content/gdrive/MyDrive/MAIC2023_공모전/ECG_adult_numpy_valid.zip', 'r') as zipObj:
    listOfFileNames = zipObj.namelist()
    for fileName in listOfFileNames:
        if fileName.endswith('npy'):
            zipRead = zipObj.read(fileName)
            curr_df = np.load(BytesIO(zipRead))
            av_data.append(curr_df)
            av_filename.append(fileName)

In [ ]:

at_filename = []  ##파일이름 리스트
at_data=[]        ##데이터 리스트
with ZipFile('/content/gdrive/MyDrive/MAIC2023_공모전/ECG_adult_numpy_train.zip', 'r') as zipObj:
    listOfFileNames = zipObj.namelist()
    for fileName in listOfFileNames:
        if fileName.endswith('npy'):
            zipRead = zipObj.read(fileName)
            curr_df = np.load(BytesIO(zipRead))
            at_data.append(curr_df)
            at_filename.append(fileName)


#데이터 관찰

In [ ]:
#print("child train 데이터 : " ,len(ct_data))
#print("adult train 데이터 : " ,len(at_data))

In [ ]:
'''
print(ct_filename)
print(len(ct_data))
print(len(ct_data[2]))
print(ct_data)
print(ct_data[2])
'''

'\nprint(ct_filename)\nprint(len(ct_data))\nprint(len(ct_data[2]))\nprint(ct_data)\nprint(ct_data[2])\n'

In [ ]:
'''
print(len(ct_data[0]))
print(ct_data[0].shape)
'''

'\nprint(len(ct_data[0]))\nprint(ct_data[0].shape)\n'

#데이터 reshape

In [ ]:
for i in range(len(ct_data)):
    ct_data[i] = ct_data[i].reshape(5000, 12)

for i in range(len(cv_data)):
    cv_data[i] = cv_data[i].reshape(5000, 12)

In [ ]:
for i in range(len(av_data)):
    av_data[i] = av_data[i].reshape(5000, 12)

for i in range(len(at_data)):
    at_data[i] = at_data[i].reshape(5000, 12)


In [ ]:
#ct_data[1]

#ECG 데이터와 csv데이터 연결

In [ ]:
new_list = [item.replace('.npy', '') for item in ct_filename]
new_list2 = [item.replace('.npy', '') for item in cv_filename]

In [ ]:
new_list3 = [item.replace('.npy', '') for item in av_filename]
new_list4 = [item.replace('.npy', '') for item in at_filename]
new_list4 = [item.replace('ECG_adult_numpy_train/', '') for item in new_list4]

In [ ]:
combined_data = pd.DataFrame({'FILENAME': new_list, 'ECG_SIGNAL': ct_data})
combined_data = pd.merge(ECG_child_age_train, combined_data, on='FILENAME', how='inner')
combined_data = combined_data[combined_data['GENDER'] != 'unknown']
combined_data["GENDER"] = combined_data['GENDER'].map({'MALE' :0, 'FEMALE':1})
combined_data

,FILENAME,GENDER,AGE,ECG_SIGNAL
0,ecg_child_0,0,1.500000,"[[76.0, 76.0, 76.0, 76.0, 75.0, 74.0, 73.0, 72..."
1,ecg_child_1,0,4.833333,"[[-10.0, -10.0, -10.0, -10.0, -10.0, -10.0, -1..."
2,ecg_child_2,1,5.583333,"[[-12.0, -12.0, -12.0, -12.0, -12.0, -12.0, -1..."
3,ecg_child_3,1,5.083333,"[[-14.0, -14.0, -14.0, -14.0, -16.0, -16.0, -1..."
4,ecg_child_4,0,0.166667,"[[-21.0, -21.0, -20.0, -20.0, -22.0, -22.0, -2..."
...,...,...,...,...
8776,ecg_child_8776,0,0.666667,"[[-10.0, -10.0, -10.0, -10.0, -6.0, -4.0, -2.0..."
8777,ecg_child_8777,0,0.083333,"[[-48.0, -48.0, -48.0, -48.0, -48.0, -48.0, -4..."
8778,ecg_child_8778,1,5.833333,"[[-10.0, -10.0, -10.0, -10.0, -10.0, -10.0, -1..."
8779,ecg_child_8779,1,2.250000,"[[0.0, 0.0, 0.0, 0.0, -2.0, -2.0, -2.0, -2.0, ..."


In [ ]:
combined_data2 = pd.DataFrame({'FILENAME': new_list4, 'ECG_SIGNAL': at_data})
combined_data2 = pd.merge(ECG_adult_age_train, combined_data2, on='FILENAME', how='inner')
combined_data2 = combined_data2[combined_data2['GENDER'] != 'unknown']
combined_data2["GENDER"] = combined_data2['GENDER'].map({'MALE' :0, 'FEMALE':1})
combined_data2

,FILENAME,GENDER,AGE,ECG_SIGNAL
0,ecg_adult_0,0,53,"[[12.0, 12.0, 12.0, 12.0, 11.0, 10.0, 9.0, 8.0..."
1,ecg_adult_1,0,68,"[[-2.0, -2.0, -2.0, -2.0, -4.0, -4.0, -4.0, -4..."
2,ecg_adult_2,0,68,"[[-10.0, -10.0, -10.0, -10.0, -10.0, -10.0, -1..."
3,ecg_adult_3,0,89,"[[-6.0, -6.0, -6.0, -5.0, -7.0, -7.0, -7.0, -7..."
4,ecg_adult_4,0,79,"[[2.0, 2.0, 2.0, 2.0, 4.0, 6.0, 8.0, 10.0, 8.0..."
...,...,...,...,...
34902,ecg_adult_35136,0,19,"[[-16.0, -16.0, -16.0, -16.0, -14.0, -14.0, -1..."
34903,ecg_adult_35137,1,63,"[[11.0, 12.0, 12.0, 12.0, 12.0, 13.0, 13.0, 13..."
34904,ecg_adult_35138,0,60,"[[-10.0, -10.0, -10.0, -10.0, -12.0, -12.0, -1..."
34905,ecg_adult_35139,1,60,"[[0.0, 0.0, 0.0, 0.0, -1.0, -2.0, -3.0, -4.0, ..."


In [ ]:
#combined_data = pd.concat([combined_data, combined_data2], axis=0)

In [ ]:
#combined_data_child = pd.DataFrame({'FILENAME': new_list2, 'ECG_SIGNAL': cv_data})
combined_data_adult = pd.DataFrame({'FILENAME': new_list3, 'ECG_SIGNAL': av_data})
#combined_valid = pd.concat([combined_data_child, combined_data_adult], axis=0)
#combined_valid_child = pd.merge(submission, combined_data_child, on='FILENAME', how='inner')
combined_valid_adult = pd.merge(submission, combined_data_adult, on='FILENAME', how='inner')

#combined_valid_child = combined_valid_child[combined_valid_child['GENDER'] != 'unknown']
#combined_valid_child["GENDER"] = combined_valid_child['GENDER'].map({'MALE' :0, 'FEMALE':1})

combined_valid_adult = combined_valid_adult[combined_valid_adult['GENDER'] != 'unknown']
combined_valid_adult["GENDER"] = combined_valid_adult['GENDER'].map({'MALE' :0, 'FEMALE':1})

#combined_valid_child

In [ ]:
combined_valid_adult

,FILENAME,GENDER,AGE,ECG_SIGNAL
0,ecg_adult_35141,0,NaN,"[[86.0, 86.0, 86.0, 86.0, 87.0, 88.0, 89.0, 90..."
1,ecg_adult_35142,1,NaN,"[[-4.0, -4.0, -4.0, -4.0, -6.0, -6.0, -6.0, -6..."
2,ecg_adult_35143,0,NaN,"[[82.0, 82.0, 82.0, 82.0, 92.0, 102.0, 112.0, ..."
3,ecg_adult_35144,0,NaN,"[[-6.0, -6.0, -6.0, -6.0, -6.0, -6.0, -6.0, -6..."
4,ecg_adult_35145,1,NaN,"[[2.0, 3.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0,..."
...,...,...,...,...
4370,ecg_adult_39536,1,NaN,"[[-8.0, -8.0, -8.0, -8.0, -6.0, -6.0, -6.0, -6..."
4371,ecg_adult_39537,1,NaN,"[[2.0, 2.0, 4.0, 2.0, 2.0, 2.0, 2.0, 1.0, 0.0,..."
4372,ecg_adult_39538,1,NaN,"[[26.0, 26.0, 26.0, 26.0, 27.0, 28.0, 29.0, 30..."
4373,ecg_adult_39539,1,NaN,"[[-15.0, -9.0, -1.0, 5.0, 2.0, -4.0, -2.0, -5...."


In [ ]:
'''
combined_data.to_csv('/content/gdrive/MyDrive/MAIC2023_공모전/MAIC_child_train_preprocessed.csv', index=False)
combined_valid.to_csv('/content/gdrive/MyDrive/MAIC2023_공모전/MAIC_child_valid_preprocessed.csv', index=False)
'''

"\ncombined_data.to_csv('/content/gdrive/MyDrive/MAIC2023_공모전/MAIC_child_train_preprocessed.csv', index=False)\ncombined_valid.to_csv('/content/gdrive/MyDrive/MAIC2023_공모전/MAIC_child_valid_preprocessed.csv', index=False)\n"

#ECG 데이터 shape 확인

In [ ]:
'''
import numpy as np

# .npy 파일 경로 지정
npy_file_path = '/content/gdrive/MyDrive/MAIC2023_공모전/ecg_adult_20000.npy'  # 실제 파일 경로로 변경하세요.

# 넘파이 배열로 .npy 파일 로드
loaded_data = np.load(npy_file_path)

# 로드한 데이터 사용 예제
print("로드한 데이터의 크기:", loaded_data.shape)
print("로드한 데이터 내용:\n", loaded_data)
'''

'\nimport numpy as np\n\n# .npy 파일 경로 지정\nnpy_file_path = \'/content/gdrive/MyDrive/MAIC2023_공모전/ecg_adult_20000.npy\'  # 실제 파일 경로로 변경하세요.\n\n# 넘파이 배열로 .npy 파일 로드\nloaded_data = np.load(npy_file_path)\n\n# 로드한 데이터 사용 예제\nprint("로드한 데이터의 크기:", loaded_data.shape)\nprint("로드한 데이터 내용:\n", loaded_data)\n'

In [ ]:
'''
reshape1 = loaded_data.reshape((5000, 12))
print(reshape1.shape)

reshape2 = loaded_data.reshape(5000, 12)
print(reshape2.shape)
'''

'\nreshape1 = loaded_data.reshape((5000, 12))\nprint(reshape1.shape)\n\nreshape2 = loaded_data.reshape(5000, 12)\nprint(reshape2.shape)\n'

#ECG 데이터 (5000, 12)로 reshape

In [ ]:
'''
import os
import numpy as np

# 데이터가 있는 디렉토리 경로
data_dir = '/content/gdrive/MyDrive/MAIC2023_공모전/'

# .npy 파일을 (5000, 12)의 shape로 재배열하고 덮어쓰기 모드로 저장하는 함수
def reshape_and_overwrite_npy_file(file_path):
    npy_data = np.load(file_path)

    # 원하는 형태인 (5000, 12)로 데이터를 재배열
    reshaped_data = npy_data.reshape((5000, 12))

    # 같은 파일에 덮어쓰기 모드로 저장
    np.save(file_path, reshaped_data)

# 주어진 디렉토리에서 모든 .npy 파일을 찾아 재배열하고 덮어쓰기 모드로 저장
for file_name in os.listdir(data_dir):
    if file_name.endswith('.npy'):
        file_path = os.path.join(data_dir, file_name)
        reshape_and_overwrite_npy_file(file_path)

print("모든 .npy 파일을 (5000, 12)의 shape로 재배열하여 기존 파일을 덮어썼습니다.")
'''

'\nimport os\nimport numpy as np\n\n# 데이터가 있는 디렉토리 경로\ndata_dir = \'/content/gdrive/MyDrive/MAIC2023_공모전/\'\n\n# .npy 파일을 (5000, 12)의 shape로 재배열하고 덮어쓰기 모드로 저장하는 함수\ndef reshape_and_overwrite_npy_file(file_path):\n    npy_data = np.load(file_path)\n\n    # 원하는 형태인 (5000, 12)로 데이터를 재배열\n    reshaped_data = npy_data.reshape((5000, 12))\n\n    # 같은 파일에 덮어쓰기 모드로 저장\n    np.save(file_path, reshaped_data)\n\n# 주어진 디렉토리에서 모든 .npy 파일을 찾아 재배열하고 덮어쓰기 모드로 저장\nfor file_name in os.listdir(data_dir):\n    if file_name.endswith(\'.npy\'):\n        file_path = os.path.join(data_dir, file_name)\n        reshape_and_overwrite_npy_file(file_path)\n\nprint("모든 .npy 파일을 (5000, 12)의 shape로 재배열하여 기존 파일을 덮어썼습니다.")\n'

In [ ]:
'''
# .npy 파일 경로 지정
npy_file_path = '/content/gdrive/MyDrive/MAIC2023_공모전/ecg_adult_19618.npy'  # 실제 파일 경로로 변경하세요.

# 넘파이 배열로 .npy 파일 로드
loaded_data = np.load(npy_file_path)

# 로드한 데이터 사용 예제
print("로드한 데이터의 크기:", loaded_data.shape)
print("로드한 데이터 내용:\n", loaded_data)
'''

'\n# .npy 파일 경로 지정\nnpy_file_path = \'/content/gdrive/MyDrive/MAIC2023_공모전/ecg_adult_19618.npy\'  # 실제 파일 경로로 변경하세요.\n\n# 넘파이 배열로 .npy 파일 로드\nloaded_data = np.load(npy_file_path)\n\n# 로드한 데이터 사용 예제\nprint("로드한 데이터의 크기:", loaded_data.shape)\nprint("로드한 데이터 내용:\n", loaded_data)\n'

#child_train 데이터와 ecg 데이터 연결

In [ ]:
'''
import numpy as np
import pandas as pd
import os

# CSV 파일 로드
csv_file_path = '/content/gdrive/MyDrive/MAIC2023_공모전/ECG_child_age_train.csv'
csv_data = pd.read_csv(csv_file_path)

# .npy 파일 로드 및 연결
data_dir = '/content/gdrive/MyDrive/MAIC2023_공모전/'  # .npy 파일이 있는 디렉토리
ecg_data = []

for index, row in csv_data.iterrows():
    file_name = row['FILENAME']
    file_path = os.path.join(data_dir, f'{file_name}.npy')

    # .npy 파일 로드
    ecg_signal = np.load(file_path)

    # 'AGE'와 'GENDER' 컬럼을 추가하여 연결
    ecg_data.append([file_name, ecg_signal, row['GENDER'], row['AGE']])

# 연결된 데이터를 데이터프레임으로 변환
combined_data = pd.DataFrame(ecg_data, columns=['FILENAME', 'ECG_SIGNAL', 'GENDER', 'AGE'])
'''

"\nimport numpy as np\nimport pandas as pd\nimport os\n\n# CSV 파일 로드\ncsv_file_path = '/content/gdrive/MyDrive/MAIC2023_공모전/ECG_child_age_train.csv'\ncsv_data = pd.read_csv(csv_file_path)\n\n# .npy 파일 로드 및 연결\ndata_dir = '/content/gdrive/MyDrive/MAIC2023_공모전/'  # .npy 파일이 있는 디렉토리\necg_data = []\n\nfor index, row in csv_data.iterrows():\n    file_name = row['FILENAME']\n    file_path = os.path.join(data_dir, f'{file_name}.npy')\n\n    # .npy 파일 로드\n    ecg_signal = np.load(file_path)\n\n    # 'AGE'와 'GENDER' 컬럼을 추가하여 연결\n    ecg_data.append([file_name, ecg_signal, row['GENDER'], row['AGE']])\n\n# 연결된 데이터를 데이터프레임으로 변환\ncombined_data = pd.DataFrame(ecg_data, columns=['FILENAME', 'ECG_SIGNAL', 'GENDER', 'AGE'])\n"

In [ ]:
#combined_data

In [ ]:
'''
def convert_to_tuple(arr):
    return tuple(arr)

# 'ECG_SIGNAL' 열의 각 요소를 튜플로 변환
combined_data['ECG_SIGNAL'] = combined_data['ECG_SIGNAL'].apply(convert_to_tuple)
'''

"\ndef convert_to_tuple(arr):\n    return tuple(arr)\n\n# 'ECG_SIGNAL' 열의 각 요소를 튜플로 변환\ncombined_data['ECG_SIGNAL'] = combined_data['ECG_SIGNAL'].apply(convert_to_tuple)\n"

In [ ]:
#combined_data

#adult_train 데이터와 ecg 데이터 연결

In [ ]:
'''
# CSV 파일 로드
csv_file_path2 = '/content/gdrive/MyDrive/MAIC2023_공모전/ECG_adult_age_train.csv'
csv_data2 = pd.read_csv(csv_file_path2)

# .npy 파일 로드 및 연결
data_dir = '/content/gdrive/MyDrive/MAIC2023_공모전/'  # .npy 파일이 있는 디렉토리
ecg_data2 = []

for index, row in csv_data2.iterrows():
    file_name = row['FILENAME']
    file_path = os.path.join(data_dir, f'{file_name}.npy')

    # .npy 파일 로드
    ecg_signal = np.load(file_path)

    # 'AGE'와 'GENDER' 컬럼을 추가하여 연결
    ecg_data2.append([file_name, ecg_signal, row['GENDER'], row['AGE']])

# 연결된 데이터를 데이터프레임으로 변환
combined_data2 = pd.DataFrame(ecg_data2, columns=['FILENAME', 'ECG_SIGNAL', 'GENDER', 'AGE'])
'''

"\n# CSV 파일 로드\ncsv_file_path2 = '/content/gdrive/MyDrive/MAIC2023_공모전/ECG_adult_age_train.csv'\ncsv_data2 = pd.read_csv(csv_file_path2)\n\n# .npy 파일 로드 및 연결\ndata_dir = '/content/gdrive/MyDrive/MAIC2023_공모전/'  # .npy 파일이 있는 디렉토리\necg_data2 = []\n\nfor index, row in csv_data2.iterrows():\n    file_name = row['FILENAME']\n    file_path = os.path.join(data_dir, f'{file_name}.npy')\n\n    # .npy 파일 로드\n    ecg_signal = np.load(file_path)\n\n    # 'AGE'와 'GENDER' 컬럼을 추가하여 연결\n    ecg_data2.append([file_name, ecg_signal, row['GENDER'], row['AGE']])\n\n# 연결된 데이터를 데이터프레임으로 변환\ncombined_data2 = pd.DataFrame(ecg_data2, columns=['FILENAME', 'ECG_SIGNAL', 'GENDER', 'AGE'])\n"

In [ ]:
#combined_data2

#AutoML

In [ ]:
'''
# PyCaret 라이브러리 설치
!pip install pycaret
'''

'\n# PyCaret 라이브러리 설치\n!pip install pycaret\n'

In [ ]:
# PyCaret 및 필요한 모듈 가져오기
'''
from pycaret.classification import *
import pandas as pd

reg_test_1 = setup(data=combined_data,
                   target='AGE',
                   train_size= 0.8,
                   fold=5,
                   categorical_features=['GENDER'])
'''

"\nfrom pycaret.classification import *\nimport pandas as pd\n\nreg_test_1 = setup(data=combined_data,\n                   target='AGE',\n                   train_size= 0.8,\n                   fold=5,\n                   categorical_features=['GENDER'])\n"

In [ ]:
#best = compare_models(sort='mse')

In [ ]:
#best_tune = tune_model(best)

In [ ]:
#evaluate_model(best_tune)

#LSTM 모델

In [ ]:
combined_data = combined_data.sample(frac=0.1)

In [ ]:
combined_data2 = combined_data2.sample(frac=0.1)

In [ ]:
combined_data.isnull().any()

FILENAME      False
GENDER        False
AGE           False
ECG_SIGNAL    False
dtype: bool

In [ ]:
#combined_valid.isnull().any()

FILENAME      False
GENDER        False
AGE            True
ECG_SIGNAL    False
dtype: bool

In [ ]:
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.optimizers import Adam

# 데이터 병합
#combined_data = pd.concat([ECG_child_age_train, ECG_adult_age_train], ignore_index=True)

# 'ECG_SIGNAL' 열의 넘파이 배열로 변환
#combined_data['ECG_SIGNAL'] = combined_data['ECG_SIGNAL'].apply(eval)

# 'AGE' 컬럼의 결측값을 0으로 채우기 (또는 원하는 처리 방법으로 대체)
combined_data['AGE'].fillna(0, inplace=True)

# 'ECG_SIGNAL' 데이터 정규화
scaler = MinMaxScaler()
combined_data['ECG_SIGNAL'] = combined_data['ECG_SIGNAL'].apply(lambda x: scaler.fit_transform(np.array(x).reshape(-1, 1)))

# 시계열 데이터를 학습 가능한 형태로 변환
X = np.array(combined_data['ECG_SIGNAL'].tolist())
y = combined_data['AGE'].values

# LSTM 모델 정의
model = Sequential()
model.add(LSTM(50, activation='tanh', input_shape=(X.shape[1], X.shape[2])))
model.add(Dense(1))

# 모델 컴파일
model.compile(optimizer=Adam(learning_rate=0.001), loss='mean_squared_error')

# LSTM 모델 학습
model.fit(X, y, epochs=10, batch_size=32)

# 모델 저장
model.save("/content/gdrive/MyDrive/MAIC2023_공모전/my_model.h5")

Epoch 1/10
112/112 [==============================] - 148s 1s/step - loss: 3060.0427
Epoch 2/10
112/112 [==============================] - 147s 1s/step - loss: 2252.3792
Epoch 3/10
 86/112 [======================>.......] - ETA: 34s - loss: 1833.3311

KeyboardInterrupt: ignored

In [ ]:
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.optimizers import Adam

# 데이터 병합
#combined_data2 = pd.concat([ECG_child_age_train, ECG_adult_age_train], ignore_index=True)

# 'ECG_SIGNAL' 열의 넘파이 배열로 변환
#combined_data2['ECG_SIGNAL'] = combined_data2['ECG_SIGNAL'].apply(eval)

# 'AGE' 컬럼의 결측값을 0으로 채우기 (또는 원하는 처리 방법으로 대체)
combined_data2['AGE'].fillna(0, inplace=True)

# 'ECG_SIGNAL' 데이터 정규화
scaler = MinMaxScaler()
combined_data2['ECG_SIGNAL'] = combined_data2['ECG_SIGNAL'].apply(lambda x: scaler.fit_transform(np.array(x).reshape(-1, 1)))

# 시계열 데이터를 학습 가능한 형태로 변환
X = np.array(combined_data2['ECG_SIGNAL'].tolist())
y = combined_data2['AGE'].values

# LSTM 모델 정의
model = Sequential()
model.add(LSTM(50, activation='relu', input_shape=(X.shape[1], X.shape[2])))
model.add(Dense(1))

# 모델 컴파일
model.compile(optimizer=Adam(learning_rate=0.001), loss='mean_squared_error')

# LSTM 모델 학습
model.fit(X, y, epochs=10, batch_size=32)

# 모델 저장
model.save("/content/gdrive/MyDrive/MAIC2023_공모전/my_model_adult.h5")

Epoch 1/10
  9/109 [=>............................] - ETA: 34:41 - loss: 3911.0947

In [ ]:
# combined_valid 데이터의 'ECG_SIGNAL' 열을 넘파이 배열로 변환
#combined_valid['ECG_SIGNAL'] = combined_valid['ECG_SIGNAL'].apply(eval)

# combined_valid 데이터의 'ECG_SIGNAL' 데이터 정규화
combined_valid['ECG_SIGNAL'] = combined_valid['ECG_SIGNAL'].apply(lambda x: scaler.fit_transform(np.array(x).reshape(-1, 1)))

# combined_valid 데이터를 시계열 데이터로 변환
X_submission = np.array(combined_valid['ECG_SIGNAL'].tolist())

# AGE 예측
predicted_age = model.predict(X_submission)

# 예측 결과를 'AGE' 컬럼에 채우기
combined_valid['AGE'] = predicted_age

# 결과 저장
combined_valid.to_csv('/content/gdrive/MyDrive/MAIC2023_공모전/submission_with_age.csv', index=False)